In [52]:
import pandas as pd
import numpy as np

In [53]:
from sklearn.metrics import mean_squared_error

In [54]:
import os

In [55]:
y_true = pd.read_csv('datasets/test_set.csv')['cpi_pct'].to_numpy()

In [56]:
prediction = []
for file_name in os.listdir('predictions/'):
    y_pred = pd.read_csv(f'predictions/{file_name}', index_col=0).to_numpy()
    
    name = file_name.rsplit('.', 1)[0]
    rmse =  mean_squared_error(y_true, y_pred, squared=False)
    print(name, "rmse:", round(rmse, 7))
    prediction.append({'model': name, 'rmse:': rmse})

benchmark1_mean rmse: 0.0032184
benchmark2_prev rmse: 0.0026672
bma rmse: 0.00245
lasso_bic rmse: 0.0022538
lasso_cv rmse: 0.0021016
lasso_cvshuffled rmse: 0.0019987
lasso_ebic rmse: 0.0022172
ols rmse: 0.0030478
pca_bma rmse: 0.0029972
pca_lasso_bic rmse: 0.0024414
pca_lasso_cv rmse: 0.003326
pca_lasso_cvshuffled rmse: 0.0024578
pca_lasso_ebic rmse: 0.0030582
pca_ols rmse: 0.0043398
pca_ridge_cv rmse: 0.0032522
random_forrest rmse: 0.00201
ridge_cv rmse: 0.0021375


In [57]:
pd.DataFrame(prediction)

,model,rmse:
0,benchmark1_mean,0.003218
1,benchmark2_prev,0.002667
2,bma,0.002450
3,lasso_bic,0.002254
4,lasso_cv,0.002102
5,lasso_cvshuffled,0.001999
6,lasso_ebic,0.002217
7,ols,0.003048
8,pca_bma,0.002997
9,pca_lasso_bic,0.002441


2000-2023 model:

- benchmark1_mean rmse: 0.0026948
- benchmark2_prev rmse: 0.0034082
- bma rmse: 0.002297
- lasso_bic rmse: 0.0023243
- lasso_ebic rmse: 0.0023711
- lasso_cv rmse: 0.0027338
- lasso_cvshuffled rmse: 0.0027338
- ols rmse: 0.0279417
- ridge_cv rmse: 0.0024787

1970-2023 model:

- benchmark1_mean rmse: 0.0032184
- benchmark2_prev rmse: 0.0026672
- bma rmse: 0.0020436
- lasso_bic rmse: 0.0021458
- lasso_ebic rmse: 0.0021458
- lasso_cv rmse: 0.0021016
- lasso_cvshuffled rmse: 0.0019987
- ols rmse: 0.0030478
- ridge_cv rmse: 0.0021375